In [69]:
import pandas as pd
import useful_functions as use
import datacompy

In [2]:
npi_me = use.get_npi_to_me()

In [107]:
# og = pd.read_csv('../../Data/DataGov/DAC_NationalDownloadableFile_211109.csv', encoding='latin')
load_1 = pd.read_csv('../../Data/DataGov/address_load_national_downloadable_2021-11-15.csv')

In [4]:
og.NPI = og.NPI.astype(str)
gov = pd.merge(npi_me, og, on='NPI')

In [5]:
gov.columns = [c.strip() for c in gov.columns.values]

In [10]:
def create(gov):
    col_rename = {
            'adr_ln_1':'addr_line_1',
            'cty':'addr_city',
            'st':'addr_state',
            'adr_ln_2':'addr_line_2',
            'ME':'me#'
        }
    gov = gov.rename(columns=col_rename)
    cols = ['entity_id',
                'me#',
                'comm_id',
                'usage',
                'load_type',
                'addr_type',
                'addr_line_1',
                'addr_line_2',
                'addr_line_3',
                'addr_city',
                'addr_state',
                'addr_zip',
                'addr_plus4',
                'addr_country',
                'source',
                'adrs_id'
                ]
    gov['addr_zip'] = [use.fix_zipcode(x) for x in gov.zip]
    gov['addr_plus4'] = [x if len(str(x))==9 else '' for x in gov.zip]
    gov['load_type'] = 'A'
    gov['addr_type'] = 'OF'
    gov['source'] = 'NAT-DOWN'
    gov['entity_id'] = ''
    gov['comm_id'] = ''
    gov['usage'] = ''
    gov['addr_country'] = ''
    gov['addr_line_3'] = ''
    gov = gov.drop_duplicates(['me#','adrs_id'])[cols]
    return (gov)

In [11]:
gov = create(gov)

In [14]:
load['me#'] = use.fix_me(load['me#'])

In [24]:
load['addr_zip'] = [use.fix_zipcode(x) for x in load.addr_zip]

In [41]:
load.addr_plus4 = load.addr_plus4.fillna(0)

In [52]:
def fix_zipcode_long(num):
    num = str(num).strip().replace('.0', '')
    num = ''.join(filter(str.isdigit, num))
    if len(num) == 1:
        num = ''
    if len(num) == 8:
        num = '0' + num
    elif len(num) == 7:
        num = '00' + num
    return num

In [54]:
load.addr_plus4 = [fix_zipcode_long(x) for x in load.addr_plus4]

In [97]:
merged = pd.merge(load, gov, on=['me#',
                'addr_line_1',
                'addr_line_2',
                'addr_city',
                'addr_state',
                'addr_zip',
                'addr_plus4'], how='left', suffixes=['','_og'])[list(gov.columns)]

In [115]:
with_id = merged.drop_duplicates()

In [83]:
gov.addr_plus4 = [str(x) for x in gov.addr_plus4]

In [84]:
compare_2 = datacompy.Compare(gov, load, join_columns='me#', df1_name = 'og', df2_name = 'loaded')

In [85]:
print(compare_2.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns     Rows
0        og       16  1331043
1    loaded       16  1330936

Column Summary
--------------

Number of columns in common: 15
Number of columns in og but not in loaded: 1
Number of columns in loaded but not in og: 1

Row Summary
-----------

Matched on: me#
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 1,330,897
Number of rows in og but not in loaded: 146
Number of rows in loaded but not in og: 39

Number of rows with some compared columns unequal: 1,330,897
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 11
Number of columns compared with all values equal: 4
Total number of values which compare unequal: 6,654,542

Columns with Unequal Values or Types
------------------------------------

          Column og dtype loaded dtype  #

In [89]:
gov.drop_duplicates(subset=['adrs_id'])

,entity_id,me#,comm_id,usage,load_type,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,addr_plus4,addr_country,source,adrs_id
0,,04201040791,,,A,OF,ADMINISTRACION DE SERVICIOS MEDICOS(MANEJO DE,CENTRO MEDICO DE PUERTO RICO BO MONACILLOS,,RIO PIEDRAS,PR,00935,,,NAT-DOWN,PR009350000RIADMINDEXX601
1,,49575000010,,,A,OF,951 NW 13TH ST,SUITE 4E,,BOCA RATON,FL,33486,334862337,,NAT-DOWN,FL334862337BO951XXSTXX417
2,,49575000010,,,A,OF,840 US HWY 1,SUITE 120,,NORTH PALM BEACH,FL,33408,334083858,,NAT-DOWN,FL334083858NO840XX1XXX401
3,,49575000010,,,A,OF,2330 S CONGRESS AVE,NaN,,PALM SPRINGS,FL,33406,334067608,,NAT-DOWN,FL334067608PA2330XAVEX400
4,,02106050565,,,A,OF,7777 HENNESSY BLVD,SUITE 4000,,BATON ROUGE,LA,70808,708080306,,NAT-DOWN,LA708080306BA7777XBLVD302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348563,,06509140021,,,A,OF,110 E 59TH ST,FL 10,,NEW YORK,NY,10022,100221304,,NAT-DOWN,NY100221304NE110XXSTXX410
1348612,,03901171199,,,A,OF,50 BUCK CREEK RD,SUITE 100,,AVON,CO,81620,816205428,,NAT-DOWN,CO816205428AV50XXXRDXX404
1348625,,01105161014,,,A,OF,3009 N BALLAS RD,SUITE 360C,,SAINT LOUIS,MO,63131,631312349,,NAT-DOWN,MO631312349SA3009XRDXX401
1348656,,04720170278,,,A,OF,1840 MEDICAL CTR PKWY,SUITE 400,,MURFREESBORO,TN,37129,371293237,,NAT-DOWN,TN371293237MU1840XPKWY410


In [90]:
for row in gov.itertuples():
    if 'ND' in row.addr_plus4:
        print(row.adrs_id)

GA3050125NDGA1640XSEXX501
MI4804827NDNE58144AVEX300
FL3257050NDMI5934XDRXX501
GA3031223NDAT285XXNEXX302
FL3257050NDMI5978XDRXX500
GA3050125NDGA1640XSEXX501
FL3257050NDMI5934XDRXX501
NY1169136NDFA1919XAVEX301
MI4804827NDNE58144AVEX300
GA3053464NDDA81XXXDRXX401
NY1142224NDRO24411AVEX300
GA3050125NDGA1640XSEXX501
GA3050125NDGA1640XSEXX501
GA3050125NDGA1640XSEXX501
GA3028162NDST140XXCOUR401
GA3007638NDRO1340XRDXX401
CA9140261NDPA8110XAVEX300
GA3050125NDGA1640XSEXX501
GA3050125NDGA1640XSEXX501
GA3050125NDGA1640XSEXX501
NY1169136NDFA1919XAVEX301
NY1169136NDFA1919XAVEX301
GA3050125NDGA1640XSEXX501
NY1169136NDFA1919XAVEX301
GA3050125NDGA1640XSEXX501
MI4886435NDOK2270XRDXX302
GA3050125NDGA1640XSEXX501


In [94]:
merged.drop_duplicates()

,entity_id_x,me#,comm_id_x,usage_x,load_type_x,addr_type_x,addr_line_1,addr_line_2,addr_line_3_x,addr_city,...,source_dtm,entity_id_y,comm_id_y,usage_y,load_type_y,addr_type_y,addr_line_3_y,addr_country_y,source_y,adrs_id
0,NaN,04201040791,NaN,NaN,A,OF,ADMINISTRACION DE SERVICIOS MEDICOS(MANEJO DE,CENTRO MEDICO DE PUERTO RICO BO MONACILLOS,NaN,RIO PIEDRAS,...,2021-11-15,,,,A,OF,,,NAT-DOWN,PR009350000RIADMINDEXX601
1,NaN,49575000010,NaN,NaN,A,OF,951 NW 13TH ST,SUITE 4E,NaN,BOCA RATON,...,2021-11-15,,,,A,OF,,,NAT-DOWN,FL334862337BO951XXSTXX417
2,NaN,49575000010,NaN,NaN,A,OF,840 US HWY 1,SUITE 120,NaN,NORTH PALM BEACH,...,2021-11-15,,,,A,OF,,,NAT-DOWN,FL334083858NO840XX1XXX401
3,NaN,49575000010,NaN,NaN,A,OF,2330 S CONGRESS AVE,NaN,NaN,PALM SPRINGS,...,2021-11-15,,,,A,OF,,,NAT-DOWN,FL334067608PA2330XAVEX400
4,NaN,02106050565,NaN,NaN,A,OF,7777 HENNESSY BLVD,SUITE 4000,NaN,BATON ROUGE,...,2021-11-15,,,,A,OF,,,NAT-DOWN,LA708080306BA7777XBLVD302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333265,NaN,01001161802,NaN,NaN,A,OF,915 OLENTANGY RIVER RD,FL 5,NaN,COLUMBUS,...,2021-11-15,,,,A,OF,,,NAT-DOWN,OH432123153CO915XXRDXX402
1333266,NaN,49500170025,NaN,NaN,A,OF,2350 SCHILLINGER S RD,A,NaN,MOBILE,...,2021-11-15,,,,A,OF,,,NAT-DOWN,AL366954177MO2350XRDXX401
1333267,NaN,49500170025,NaN,NaN,A,OF,7943 MOFFETT RD,NaN,NaN,SEMMES,...,2021-11-15,,,,A,OF,,,NAT-DOWN,AL365755409SE7943XRDXX300
1333268,NaN,49500170025,NaN,NaN,A,OF,4402 OLD SHELL RD,NaN,NaN,MOBILE,...,2021-11-15,,,,A,OF,,,NAT-DOWN,AL366081912MO4402XRDXX400


In [96]:
len(load)-len(merged)

-2334

In [117]:
# none = with_id.fillna('None')
with_id[(none.adrs_id=='None')].to_csv('../../Data/DataGov/NO_ID.csv', index=False)

In [114]:
with_id.fillna('')

,entity_id,me#,comm_id,usage,load_type,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,addr_plus4,addr_country,source,adrs_id
0,None,04201040791,None,None,A,OF,ADMINISTRACION DE SERVICIOS MEDICOS(MANEJO DE,CENTRO MEDICO DE PUERTO RICO BO MONACILLOS,None,RIO PIEDRAS,PR,00935,,None,NAT-DOWN,PR009350000RIADMINDEXX601
1,None,49575000010,None,None,A,OF,951 NW 13TH ST,SUITE 4E,None,BOCA RATON,FL,33486,334862337,None,NAT-DOWN,FL334862337BO951XXSTXX417
2,None,49575000010,None,None,A,OF,840 US HWY 1,SUITE 120,None,NORTH PALM BEACH,FL,33408,334083858,None,NAT-DOWN,FL334083858NO840XX1XXX401
3,None,49575000010,None,None,A,OF,2330 S CONGRESS AVE,None,None,PALM SPRINGS,FL,33406,334067608,None,NAT-DOWN,FL334067608PA2330XAVEX400
4,None,02106050565,None,None,A,OF,7777 HENNESSY BLVD,SUITE 4000,None,BATON ROUGE,LA,70808,708080306,None,NAT-DOWN,LA708080306BA7777XBLVD302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333333,None,01001161802,None,None,A,OF,915 OLENTANGY RIVER RD,FL 5,None,COLUMBUS,OH,43212,432123153,None,NAT-DOWN,OH432123153CO915XXRDXX402
1333334,None,49500170025,None,None,A,OF,2350 SCHILLINGER S RD,A,None,MOBILE,AL,36695,366954177,None,NAT-DOWN,AL366954177MO2350XRDXX401
1333335,None,49500170025,None,None,A,OF,7943 MOFFETT RD,None,None,SEMMES,AL,36575,365755409,None,NAT-DOWN,AL365755409SE7943XRDXX300
1333336,None,49500170025,None,None,A,OF,4402 OLD SHELL RD,None,None,MOBILE,AL,36608,366081912,None,NAT-DOWN,AL366081912MO4402XRDXX400


In [110]:
load_1['me#'] = use.fix_me(load_1['me#'])

In [112]:
load_1['adrs_id']=with_id.adrs_id

In [113]:
load_1

,entity_id,me#,comm_id,usage,load_type,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,addr_plus4,addr_country,source,source_dtm,adrs_id
0,NaN,04201040791,NaN,NaN,A,OF,ADMINISTRACION DE SERVICIOS MEDICOS(MANEJO DE,CENTRO MEDICO DE PUERTO RICO BO MONACILLOS,NaN,RIO PIEDRAS,PR,935,NaN,NaN,NAT-DOWN,2021-11-15,PR009350000RIADMINDEXX601
1,NaN,49575000010,NaN,NaN,A,OF,951 NW 13TH ST,SUITE 4E,NaN,BOCA RATON,FL,33486,3.34862e+08,NaN,NAT-DOWN,2021-11-15,FL334862337BO951XXSTXX417
2,NaN,49575000010,NaN,NaN,A,OF,840 US HWY 1,SUITE 120,NaN,NORTH PALM BEACH,FL,33408,3.34084e+08,NaN,NAT-DOWN,2021-11-15,FL334083858NO840XX1XXX401
3,NaN,49575000010,NaN,NaN,A,OF,2330 S CONGRESS AVE,NaN,NaN,PALM SPRINGS,FL,33406,3.34068e+08,NaN,NAT-DOWN,2021-11-15,FL334067608PA2330XAVEX400
4,NaN,02106050565,NaN,NaN,A,OF,7777 HENNESSY BLVD,SUITE 4000,NaN,BATON ROUGE,LA,70808,7.0808e+08,NaN,NAT-DOWN,2021-11-15,LA708080306BA7777XBLVD302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330931,NaN,01001161802,NaN,NaN,A,OF,915 OLENTANGY RIVER RD,FL 5,NaN,COLUMBUS,OH,43212,432123153,NaN,NAT-DOWN,2021-11-15,PA156012282GR532XXSTXX402
1330932,NaN,49500170025,NaN,NaN,A,OF,2350 SCHILLINGER S RD,A,NaN,MOBILE,AL,36695,366954177,NaN,NAT-DOWN,2021-11-15,CA941432204SA521XXAVEX300
1330933,NaN,49500170025,NaN,NaN,A,OF,7943 MOFFETT RD,NaN,NaN,SEMMES,AL,36575,365755409,NaN,NAT-DOWN,2021-11-15,CA941432204SA505XXAVEX300
1330934,NaN,49500170025,NaN,NaN,A,OF,4402 OLD SHELL RD,NaN,NaN,MOBILE,AL,36608,366081912,NaN,NAT-DOWN,2021-11-15,CA941432204SA505XXAVEX302


In [173]:
og[og[' adr_ln_1']=='5978 BERRYHILL MEDICAL PARK DR'][[' adr_ln_1',' adr_ln_2', ' adrs_id']]

,adr_ln_1,adr_ln_2,adrs_id
517689,5978 BERRYHILL MEDICAL PARK DR,NaN,FL3257050NDMI5978XDRXX500


In [176]:
manual = pd.read_csv('../../Data/DataGov/NO_ID.csv')

In [179]:
all_id = pd.concat([with_id[(none.adrs_id!='None')],manual])

In [180]:
all_id.to_csv('../../Data/DataGov/address_load_national_downloadable_with_id_2021-11-15.csv', index=False)

In [184]:
og[' phn_numbr']

0                   NaN
1                   NaN
2                   NaN
3          9.178286e+09
4          7.035910e+09
               ...     
2352941    3.033214e+09
2352942    9.016858e+09
2352943    9.016858e+09
2352944    9.072780e+09
2352945    9.072780e+09
Name:  phn_numbr, Length: 2352946, dtype: float64